In [18]:
!pip install lxml


  Obtaining dependency information for lxml from https://files.pythonhosted.org/packages/78/8d/96b95d704fab4a95651ceeb6022855ae5a3c631f86c6647749a2e868af92/lxml-4.9.3-cp310-cp310-macosx_11_0_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 6.2 MB/s eta 0:00:0000:0100:01


In [27]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import country_converter as coco

## Get the data
The function bellow is used to extract the first table present in a wikipedia Page

In [28]:

def getDataFrameFromWikipedia(wikipedia_url, table_no=1):
    """
    returns a dataframe of the data from the wikipedia page. \n
    optionaly, if there are multiple tables on the page, you can specify which table to use in `table_no`.
    """
    response = requests.get(wikipedia_url)
    print(response.status_code)
    soup = BeautifulSoup(response.text, 'html.parser')
    tables = soup.findAll('table', {'class': 'wikitable'})
    
    if table_no >= 1 and table_no <= len(tables):
        df = pd.read_html(str(tables[table_no - 1]))[0]
        return df
    else:
        return None

## Normalise the country
- As there are no standerdised way to get countries, we use a library called `country_converter` to convert countries for us.
- Some country might not exist (Channel Islands, European Union, World...), in that case, we just drop the whole record for this country

In [29]:
def normaliseCountryNames(df):
    df.columns.values[0] = 'Country'
    df['Country'] = df['Country'].apply(lambda x: coco.convert(names=x, to='name_short', not_found='not found'))
    df = df[df['Country'] != 'not found']
    return df

## All column

In [30]:
the_final_table = []

In [31]:
## GET GDP PER CAPITA

gdp_per_capita = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)_per_capita")

df= gdp_per_capita

df = df.drop(["IMF[4][5]", "United Nations[7]"], axis = 1)
df = df.drop(0)
df.columns = df.columns.droplevel(-1)
df.columns = [*df.columns[:-1], 'years']
df.columns.values[0] = 'Country'

df = df.drop(["UN Region", "years"], axis = 1)


df = normaliseCountryNames(df)

the_final_table = df

print(df.head(10000)) 

200


/var/folders/p_/y91nv_9n4pd_cdyjthbpxm900000gn/T/ipykernel_12771/3126715534.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]
/var/folders/p_/y91nv_9n4pd_cdyjthbpxm900000gn/T/ipykernel_12771/563173382.py:7: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df = df.drop(["IMF[4][5]", "United Nations[7]"], axis = 1)
Channel Islands not found in regex
European Union[n 1] not found in regex
World not found in regex
Zanzibar not found in regex


           Country World Bank[6]
1           Monaco        234317
2    Liechtenstein        184083
3       Luxembourg        126426
4          Bermuda        118846
5          Ireland        104039
..             ...           ...
219     Madagascar           505
220    South Sudan          1072
221   Sierra Leone           461
222    Afghanistan           364
223        Burundi           238

[219 rows x 2 columns]


In [32]:
# List of countries by Internet connection speeds, Fixed broadband, Average download speed (Mbit/s) (Ookla).

internet_speed = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_countries_by_Internet_connection_speeds")

df = internet_speed
df = df.drop(["Rank", "Averagedownloadspeed(Mbit/s)(M-Lab)[2]", "Averagedownloadspeed(Mbit/s)(SpeedTestNet.io)", "Averagedownloadspeed(Mbit/s)(Speed-Test-Pros.com)[4]"], axis = 1)
df.columns = [*df.columns[:-1], "Average download speed (Mbit/s)"]
df = normaliseCountryNames(df)

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')

print(df.head(10000)) 

200


/var/folders/p_/y91nv_9n4pd_cdyjthbpxm900000gn/T/ipykernel_12771/3126715534.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]


             Country  Average download speed (Mbit/s)
0            Romania                           178.90
1        South Korea                           241.58
2          Hong Kong                           265.17
3             Monaco                           220.35
4          Singapore                           259.11
..               ...                              ...
96          Dominica                            37.62
97   North Macedonia                            37.20
98            Greece                            36.73
99        Bangladesh                            36.02
100          Senegal                            35.32

[101 rows x 2 columns]


In [33]:
# List of countries by alcohol consumption per capita, Recorded per capita consumption of pure alcohol (litres) per adult 15 years of age and over per year, 2016.
alcohol_consumption = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_countries_by_alcohol_consumption_per_capita")

df = alcohol_consumption

df = df.drop(["1996[7]"], axis = 1)
df.columns = [*df.columns[:-1], "Alcohiol consumption per capita (litres) 2016"]
df = normaliseCountryNames(df)

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')

print(df.head(10000))

200


/var/folders/p_/y91nv_9n4pd_cdyjthbpxm900000gn/T/ipykernel_12771/3126715534.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]
Netherlands Antilles not found in regex


         Country Alcohiol consumption per capita (litres) 2016
0    Afghanistan                                           0.2
1        Albania                                           7.5
2        Algeria                                           0.9
3        Andorra                                          11.3
4         Angola                                           6.4
..           ...                                           ...
186    Venezuela                                           5.6
187      Vietnam                                           8.3
188        Yemen                                           0.1
189       Zambia                                           4.8
190     Zimbabwe                                           4.8

[190 rows x 2 columns]


In [34]:
# List of countries by intentional homicide rate, Intentional homicide victims per 100,000 inhabitants. From UNODC, rate.

homicide_rate = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_countries_by_intentional_homicide_rate",2)

df = homicide_rate

df = df.drop(["Region", "Subregion", "Year", "Count"], axis = 1)
df.columns = [*df.columns[:-1], "Intentional homicide victims per 100,000 inhabitants"]
df['Location'] = df['Location'].str.replace('*', '', regex=False).str.strip()
df = df.rename(columns={'Location': 'Country'})
df = normaliseCountryNames(df)

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')

print(df.head(10000))

200


/var/folders/p_/y91nv_9n4pd_cdyjthbpxm900000gn/T/ipykernel_12771/3126715534.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]
Northern Ireland not found in regex
Scotland not found in regex


            Country  Intentional homicide victims per 100,000 inhabitants
0       Afghanistan                                                4.0   
1           Albania                                                2.3   
2           Algeria                                                1.6   
3    American Samoa                                                0.0   
4           Andorra                                                2.6   
..              ...                                                ...   
201       Venezuela                                               19.3   
202         Vietnam                                                1.5   
203           Yemen                                                6.3   
204          Zambia                                                5.2   
205        Zimbabwe                                                6.1   

[204 rows x 2 columns]


In [35]:
#11. Islam by country, Muslim percentage of total population
islam_by_country = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/Islam_by_country")
df = islam_by_country
df = df.drop(["Total Population", "Muslim Population", "Percentage of world (%)", "Sources"], axis = 1)
df = df.drop(["Unnamed: 6"], axis = 1)
df = df.rename(columns={'Country/Region': 'Country'})
df = normaliseCountryNames(df)
df = df.rename(columns={'Muslim percentage of total population': 'Muslim percentage of total population_islam'})

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')
print(df.head(10000))

200


/var/folders/p_/y91nv_9n4pd_cdyjthbpxm900000gn/T/ipykernel_12771/3126715534.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]
Netherlands Antilles not found in regex


                       Country Muslim percentage of total population_islam
0                  Afghanistan                                        99.7
1                      Albania                                        58.8
2                      Algeria                                          99
3               American Samoa                                       < 0.1
4                      Andorra                                         2.6
..                         ...                                         ...
227  Wallis and Futuna Islands                                       < 0.1
228             Western Sahara                                        99.4
229                      Yemen                                        97.2
230                     Zambia                                           1
231                   Zimbabwe                                         0.7

[231 rows x 2 columns]


In [36]:
#12 Buddhism by country, Buddhist population by country, % population.
Buddhist_by_country = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/Buddhism_by_country")

df = Buddhist_by_country
df = df.drop(["Pew estimates (2010)[1]", "Other estimates"], axis = 1)
df.columns = df.columns.droplevel(0)
df = df.drop(["Population", "No. of Buddhists", "Census Year"], axis = 1)
#print(df.columns)

df = df.rename(columns={'Country/Territory': 'Country'})
df = df.rename(columns={'% Buddhist': 'Buddhist population by country, % population'})
df = normaliseCountryNames(df)

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')

print(df.head(10000))

200


/var/folders/p_/y91nv_9n4pd_cdyjthbpxm900000gn/T/ipykernel_12771/3126715534.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]
/var/folders/p_/y91nv_9n4pd_cdyjthbpxm900000gn/T/ipykernel_12771/3145737077.py:5: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df = df.drop(["Pew estimates (2010)[1]", "Other estimates"], axis = 1)
World not found in regex


            Country Buddhist population by country, % population
0       Afghanistan                                          NaN
1           Albania                                          NaN
2           Algeria                                          NaN
3    American Samoa                                          NaN
4           Andorra                                          NaN
..              ...                                          ...
174       Venezuela                                          NaN
175         Vietnam                                          NaN
176           Yemen                                          NaN
177          Zambia                                          NaN
178        Zimbabwe                                          NaN

[179 rows x 2 columns]


In [37]:
#13 Jewish population by country, Table, Core population, pct
Jewish_by_country = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/Jewish_population_by_country")
df = Jewish_by_country

df = df.drop(["Connected population", "Enlarged population","Eligible population","National official"], axis = 1)

df.columns = df.columns.droplevel(0)
df = df.drop(["Total", "pmp"], axis = 1)
df = df.rename(columns={'Countries': 'Country'})
df = df.rename(columns={'% pct': 'Core population, pct'})
#print(df.columns)

df = normaliseCountryNames(df)


the_final_table = pd.merge(the_final_table, df, on='Country', how='outer', suffixes=('_final', '_jewish'))


print(df.head(10000))

200


/var/folders/p_/y91nv_9n4pd_cdyjthbpxm900000gn/T/ipykernel_12771/3126715534.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]
/var/folders/p_/y91nv_9n4pd_cdyjthbpxm900000gn/T/ipykernel_12771/2438867356.py:5: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df = df.drop(["Connected population", "Enlarged population","Eligible population","National official"], axis = 1)
Netherlands Antilles not found in regex
World not found in regex


                    Country  pct
0                    Israel   39
1             United States   35
2                    France  2.8
3                 Palestine  2.7
4                    Canada  2.4
..                      ...  ...
107                 Iceland    —
108              Montenegro    —
109  British Virgin Islands    —
110           Liechtenstein    —
111        Falkland Islands    —

[111 rows x 2 columns]


In [38]:
#14 List of countries by infant and under-five mortality rates,  Under-five mortality (deaths/1,000 live births) – 2019 estimates
#mortality_rate = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_countries_by_infant_and_under-five_mortality_rates",4)

#df = mortality_rate


#df = df.drop(["Connected population", "Enlarged population","Eligible population","National official"], axis = 1)
#print(df.columns)

In [76]:
#16 List of countries by minimum wage, List, Annual, US$.
list = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_countries_by_minimum_wage")
df = list

df = df.drop(["Minimum wage", "Workweek (hours)[4]","Hourly","Effective per"], axis = 1)


df.columns = df.columns.droplevel(0)

df = df.drop(["PPP (Int$)[7]"], axis = 1)
df = df.drop(0)
df = df.iloc[:, :-1]

df = df.rename(columns={'Country': 'Country'})
df = df.rename(columns={'Nominal (US$)[6]': 'Annual US$'})
#print(df.columns)

df = normaliseCountryNames(df)


the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')


print(df.head(10000))


200


/var/folders/p_/y91nv_9n4pd_cdyjthbpxm900000gn/T/ipykernel_12771/3126715534.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]
/var/folders/p_/y91nv_9n4pd_cdyjthbpxm900000gn/T/ipykernel_12771/4045473643.py:5: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df = df.drop(["Minimum wage", "Workweek (hours)[4]","Hourly","Effective per"], axis = 1)


         Country  Annual US$
1    Afghanistan      858.00
2        Albania     4637.00
3        Algeria     1777.00
4        Andorra    18253.00
5         Angola      663.00
..           ...         ...
197    Venezuela       10.32
198      Vietnam     1591.00
199        Yemen         NaN
200       Zambia      596.00
201     Zimbabwe         NaN

[201 rows x 2 columns]


In [99]:
#18 List of countries by income equality, UN, World Bank and CIA list – income ratios and Gini indices, WB Gini %.
list = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_countries_by_income_equality")
df = list

df = df.drop(["Subregion", "Region","UN R/P","CIA R/P[4]"], axis = 1)
df.columns = df.columns.droplevel(0)
df = df.drop(["Year"], axis = 1)
df = df.drop(0)


df = df.rename(columns={'Country': 'Country'})
df = df.rename(columns={'%': 'WB Gini %'})
df = normaliseCountryNames(df)

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')


print(df.head(10000))

200


/var/folders/p_/y91nv_9n4pd_cdyjthbpxm900000gn/T/ipykernel_12771/3126715534.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]
/var/folders/p_/y91nv_9n4pd_cdyjthbpxm900000gn/T/ipykernel_12771/2863879547.py:5: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df = df.drop(["Subregion", "Region","UN R/P","CIA R/P[4]"], axis = 1)
European Union not found in regex
World not found in regex


       Country  WB Gini %
1      Albania       29.4
2      Algeria       27.6
3       Angola       51.3
4    Argentina       42.0
5      Armenia       27.9
..         ...        ...
165  Venezuela       44.8
166    Vietnam       36.8
167      Yemen       36.7
168     Zambia       55.9
169   Zimbabwe       50.3

[168 rows x 2 columns]


In [106]:
#19. List of countries by total health expenditure per capita, Total health expenditure per capita in 2018 PPP international U.S. dollars, 2018.
list = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_countries_by_total_health_expenditure_per_capita",2)
df = list

df = df.drop(["2019", "2020","2021"], axis = 1)
df = df.rename(columns={'Location': 'Country'})
df = df.rename(columns={'2018': 'Total health 2018'})

df = normaliseCountryNames(df)

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')


print(df.head(10000))

200


/var/folders/p_/y91nv_9n4pd_cdyjthbpxm900000gn/T/ipykernel_12771/3126715534.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]
Congo * not found in regex


                 Country  Total health 2018
0            Afghanistan                 71
1                Algeria                258
2                Andorra               3188
3                 Angola                 84
4    Antigua and Barbuda                850
..                   ...                ...
183              Vanuatu                102
184            Venezuela                165
185              Vietnam                164
186               Zambia                 75
187             Zimbabwe                115

[187 rows x 2 columns]


In [110]:
#20 List of countries by suicide rate, Suicide rates by sex and country, All.
list_suicide  = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_countries_by_suicide_rate")
df = list_suicide

df = df.drop(["Male", "Female"], axis = 1)

df = df.rename(columns={'Country': 'Country'})
df = df.rename(columns={'All': 'All Suicide rates by sex and country'})

df = normaliseCountryNames(df)

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')


print(df.head(10000))

200


/var/folders/p_/y91nv_9n4pd_cdyjthbpxm900000gn/T/ipykernel_12771/3126715534.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]
Africa not found in regex
Americas not found in regex
South-East Asia not found in regex
Europe not found in regex
Eastern Mediterranean not found in regex
Western Pacific not found in regex
Global not found in regex


                 Country  All Suicide rates by sex and country
0            Afghanistan                                   6.0
1                Albania                                   3.7
2                Algeria                                   2.6
3                 Angola                                  12.6
4    Antigua and Barbuda                                   0.3
..                   ...                                   ...
178            Venezuela                                   2.1
179              Vietnam                                   7.2
180                Yemen                                   7.1
181               Zambia                                  14.4
182             Zimbabwe                                  23.6

[183 rows x 2 columns]


In [92]:
#17 List of countries by external debt, List of countries with respect to external debt, % GDP
list_countries = getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/List_of_countries_by_external_debt")
df = list_countries

df['Country'] = df['Country'].astype(str)

df = df.drop(["Per capita US dollars", "External debt US dollars","% of total wealth[1]","% of total wealth[1]","Date"], axis = 1)

df = df.rename(columns={'Country/Region': 'Country'})
df = df.rename(columns={'of GDP': 'external debt % GDP'})


df['% of GDP'] = df['% of GDP'].round(2)
print(df)

df = normaliseCountryNames(df)


the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')


print(df.head(10000))


200


/var/folders/p_/y91nv_9n4pd_cdyjthbpxm900000gn/T/ipykernel_12771/3126715534.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]


KeyError: 'Country'

In [65]:
#15 Age of criminal responsibility, By country, age reduced

Age= getDataFrameFromWikipedia("https://en.wikipedia.org/wiki/Age_of_criminal_responsibility")

df = Age


df = df.drop(["Age (full)[b]", "Ref","Notes"], axis = 1)

df = df.rename(columns={'Country': 'Country'})
df = df.rename(columns={'Age (reduced)[a]': 'By country, age reduced'})


#df['By country, age reduced'] = df['By country, age reduced'].apply(tuple)


#print("Unique values in 'By country, age reduced':")
#print(df['By country, age reduced'].unique())

#df['By country, age reduced'] = df['By country, age reduced'].apply(lambda x: ''.join(map(str, x)))

#the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')

# ...
df['By country, age reduced'] = df['By country, age reduced'].apply(lambda x: ''.join(map(str, x)))

the_final_table = pd.merge(the_final_table, df, on='Country', how='outer')
print(the_final_table)
print(df.head(10000))


200


/var/folders/p_/y91nv_9n4pd_cdyjthbpxm900000gn/T/ipykernel_12771/3126715534.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(tables[table_no - 1]))[0]


MergeError: Passing 'suffixes' which cause duplicate columns {'By country, age reduced_x'} is not allowed.

In [46]:
print(the_final_table.head(10000))

the_final_table.to_csv('the_final_table.csv', index=False)


                   Country World Bank[6]  Average download speed (Mbit/s)  \
0                   Monaco        234317                           220.35   
1            Liechtenstein        184083                           193.79   
2               Luxembourg        126426                           154.26   
3                  Bermuda        118846                              NaN   
4                  Ireland        104039                           112.81   
..                     ...           ...                              ...   
241          Guinea Bissau           NaN                              NaN   
242             Kyrgyzstan           NaN                              NaN   
243  São Tomé and Príncipe           NaN                              NaN   
244              Swaziland           NaN                              NaN   
245                 Turkey           NaN                              NaN   

    Alcohiol consumption per capita (litres) 2016  \
0                     